<a href="https://colab.research.google.com/github/AlexEskin/Templates/blob/master/Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf crops_train
!rm -rf crops_test
!rm -rf maps_resized
!rm -rf examples

!mkdir maps_resized
!mkdir crops_train
!mkdir crops_test
!mkdir examples

import os
import cv2
import numpy as np
import os
from google.colab import files

WIDTH = 256
HEIGHT = 256
SOURCE = 'MapsExample'

def resizeImages():
    path_a = SOURCE
    path_b = 'maps_resized'

    filenames = []
    for r, d, f in os.walk(path_a):
        for file in f:
            if ".png" in file:
               filenames.append(file)
    print(filenames)
    images = []
    minSizeX = None
    minSizeY = None

    for filename in filenames:
        f_a = os.path.join(path_a, filename)
        print(f_a)
        im_a = cv2.imread(f_a)
        if minSizeX != None:
            minSizeX = min(minSizeX,im_a.shape[0])
        else:
            minSizeX = im_a.shape[0]
        if minSizeY != None:
            minSizeY = min(minSizeY,im_a.shape[1])          
        else:
            minSizeY = im_a.shape[1]
    print(minSizeX,minSizeY)
    for filename in filenames:
        f_a = os.path.join(path_a, filename)
        f_b = os.path.join(path_b, filename)

        im_a = cv2.imread(f_a)
        crop_img = im_a[0:minSizeX, 0:minSizeY]
        cv2.imwrite(f_b,crop_img)

def generateCrops():
    path_a = 'maps_resized'
    path_b = 'crops_train'
    path_c = 'crops_test'
    examples = 'examples'
    filenames = []
    for r, d, f in os.walk(path_a):
        for file in f:
            filenames.append(file)
    counter = 0
    for filename in filenames:
        f_a = os.path.join(path_a, filename)
        im_a = cv2.imread(f_a)
        shape = im_a.shape
        examples_counter = 0
        for i in range(0,shape[0]-WIDTH-1,64):
            for j in range(0,shape[1]-HEIGHT-1,64):
                path = path_b
                if not counter%7:
                    path = path_c       
                submap = os.path.join(path, filename)
                try:
                  os.mkdir(submap)
                except:
                  pass
                submap = os.path.join(submap,str(counter)+".png")
                submap_image = im_a[i:i+HEIGHT,j:j+WIDTH]
                cv2.imwrite(submap,submap_image)
                if examples_counter < 32:
                    expath = os.path.join(examples, filename)
                    try:
                        if not examples_counter:
                            os.mkdir(expath)
                    except Exception as e:
                        pass
                    exmap = os.path.join(expath,str(counter)+".png")
                    cv2.imwrite(exmap,submap_image)
                    examples_counter = examples_counter + 1
                counter += 1

"""Download maps"""

!rm -rf MapsExample

!git clone https://github.com/AlexEskin/MapsExample.git

!ls MapsExample

resizeImages()
generateCrops()


Cloning into 'MapsExample'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 55 (delta 0), reused 55 (delta 0), pack-reused 0
Unpacking objects: 100% (55/55), done.
map_10.png  map_19.png	map_27.png  map_35.png	map_43.png  map_5.png
map_11.png  map_1.png	map_28.png  map_36.png	map_44.png  map_6.png
map_12.png  map_20.png	map_29.png  map_37.png	map_45.png  map_7.png
map_13.png  map_21.png	map_2.png   map_38.png	map_46.png  map_8.png
map_14.png  map_22.png	map_30.png  map_39.png	map_47.png  map_9.png
map_15.png  map_23.png	map_31.png  map_3.png	map_48.png  README.md
map_16.png  map_24.png	map_32.png  map_40.png	map_49.png
map_17.png  map_25.png	map_33.png  map_41.png	map_4.png
map_18.png  map_26.png	map_34.png  map_42.png	map_50.png
['map_50.png', 'map_31.png', 'map_29.png', 'map_23.png', 'map_12.png', 'map_14.png', 'map_1.png', 'map_21.png', 'map_34.png', 'map_2.png', 'map_42.png', 'map_

In [2]:
import tensorflow as tf
import numpy as np
import datetime
import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')

import pathlib
import random
import numpy as np

CLASS_CNT = None

def generate_dataset(dir):
  global CLASS_CNT
  data_root = pathlib.Path(dir)
  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  if CLASS_CNT == None:
    CLASS_CNT = len(label_names)
    
  all_image_paths = list(data_root.glob('*/*'))
  all_image_paths = [str(path) for path in all_image_paths]
  random.shuffle(all_image_paths)

  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  label_to_index = dict((name, index) for index,name in enumerate(label_names))

  all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                      for path in all_image_paths]
  print("First 10 labels indices: ", all_image_labels[:10])

  return (all_image_paths,all_image_labels)

dataset = generate_dataset("crops_train")
test_dataset = generate_dataset("crops_test")

def tr_generator(n):
  num = 0
  id = random.randint(0,len(dataset[0])*65535)
  while num < n:
    id %= len(dataset[0])
    img = cv2.imread(dataset[0][id],cv2.IMREAD_GRAYSCALE)
    img = np.expand_dims(img, -1).astype(np.float32)
    yield img*(1/255.0), dataset[1][id]
    num += 1
    id += 1
    
    
def t_generator(n):
  num = 0
  id = random.randint(0,len(dataset[0])*65535)
  while num < n:
    id %= len(test_dataset[0])
    img = cv2.imread(test_dataset[0][id],cv2.IMREAD_GRAYSCALE)
    img = np.expand_dims(img, -1).astype(np.float32)
    yield img*(1/255.0), test_dataset[1][id]
    num += 1
    id += 1
    

input_data = dataset[0][0]
input_data = cv2.imread(input_data,cv2.IMREAD_GRAYSCALE).astype(np.float32)
input_data = np.expand_dims(input_data, -1)
input_data_shape = input_data.shape


First 10 labels indices:  [46, 16, 17, 12, 1, 37, 35, 31, 2, 5]
First 10 labels indices:  [13, 9, 31, 14, 39, 0, 26, 47, 46, 4]


In [3]:
import os

strategy = None
if 'COLAB_TPU_ADDR' in os.environ:
  resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.contrib.distribute.initialize_tpu_system(resolver)
  strategy = tf.contrib.distribute.TPUStrategy(resolver)

def create_model_vgg19():
  model = tf.keras.models.Sequential()
  drop = 0.2
  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))  
  model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1024))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))  
  model.add(tf.keras.layers.Dropout(drop))
  model.add(tf.keras.layers.Dense(CLASS_CNT))
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()
  return model

def create_model():
  model = tf.keras.models.Sequential()
  drop = 0.2
  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(64, (9, 9), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(128, (9, 9), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))

  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(96, (7, 7), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(192, (7, 7), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(drop))

  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))

  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))
  
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1024))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))  
  model.add(tf.keras.layers.Dropout(drop))
  model.add(tf.keras.layers.Dense(CLASS_CNT))
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()
  return model

if strategy != None:
  with strategy.scope():
    model = create_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adadelta( ),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])
else:
    model = create_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adadelta( ),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Initializing the TPU system.
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.46.87.18:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12031864767707300793)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12463054145494924263)
INFO:tensorflow:*** 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

while True:
  training_generator = tr_generator(4160)
  model.load_weights('/content/drive/My Drive/fashion_mnist.h5')
  # validation_generator = t_generator(200)
  model.save_weights('/content/drive/My Drive/fashion_mnist.h5', overwrite=True)
  train_xy =list(zip(*training_generator))
  print("train ready")
  # test_xy = list(zip(*validation_generator))
  # print("validation ready")
  train_xy = (np.array(train_xy[0]).astype(np.float32),np.array(train_xy[1]).astype(np.float32))
  # test_xy = (np.array(test_xy[0]).astype(np.float32),np.array(test_xy[1]).astype(np.float32))
  print("fitting")
  history = model.fit(
      train_xy[0],train_xy[1],
      epochs=75,
      steps_per_epoch=13
  )
  
  print(datetime.datetime.now())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
train ready
fitting
Epoch 1/75
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
13/13 [==============================] - 38s 3s/step - loss: 0.7485 - sparse_categorical_accuracy: 0.7399
Epoch 2/75
13/13 [==============================] - 21s 2s/step - loss: 0.7029 - sparse_categorical_accuracy: 0.7522
Epoch 3/75
13/13 [==============================] - 21s 2s/step - loss: 0.6878 - sparse_categorical_accuracy: 0.7603
Epoch

In [0]:
files.download('fashion_mnist.h5')